<a id='top_cell'></a>

# Advanced Python - Reading YAML configuration files
<div style="text-align: right"> [Back to Start](0 Start.ipynb) </div>

YAML (YAML Ain't Markup Language) is a human-readable serialization language. You can compare it with JSON (JavaScript Object Notation), but it is much more readable. YAML is mainly used for reading configuration files. Like Python, YAML syntax is rather strict and relies a lot on proper indentation.

Official documentation: [YAML Ain't Markup Language](http://yaml.org), [](https://yaml.readthedocs.io/en/latest/)

Other implementations: 

* [PyYAML module](http://pyyaml.org/wiki/PyYAMLDocumentation), use ``pip install pyyaml``
* [ruamel.yaml](https://yaml.readthedocs.io/en/latest/), use ``pip install ruamel.yaml``
 

In [ ]:
import logging

Let's first have a look at what we often find in Python code, hardcoded values of all kind.

In [ ]:
server = {
    'protocol': "https://",
    'hostname': "apod.nasa.gov",
    'location': "/apod/astropix.html",
    'search'  : "/cgi-bin/apod/apod_search"    
}

use_anonymous = True

In [ ]:
# value                    name                          unit          reference
#===============================================================================
cc     = 299792458.        # speed of light              m/s
cc_air = cc/1.0002773      # speed of light in air       m/s           T=15C, standard atmospheric pressure
Msol   = 1.988547e30       # solar mass                  kg            Harmanec & Prsa 2011
Rsol   = 6.95508e8         # solar radius                m             Harmanec & Prsa 2011
Lsol   = 3.846e26          # solar luminosity            W             Harmanec & Prsa 2011
Tsol   = 5779.5747         # solar effective temperature K             Harmanec & Prsa 2011

cc_units     = 'm s-1'
cc_air_units = 'm s-1'
Msol_units   = 'kg'
Rsol_units   = 'm'
Lsol_units   = 'kg m2 s-3'
Lsol_units   = 'kg m2 s-3'
Tsol_units   = 'K'

In [ ]:
obs_par = {
    "MissionDuration":  6.5,
    "NumExposures":     10,
    "BeginExposureNr":  0,
    "ExposureTime":     21,
    "RApointing":       180,
    "DecPointing":      -70,
    "Fluxm0":           1.00179e8,
    "StarCatalogFile":  "inputfiles/starcatalog.txt"
}

---

In [ ]:
print (f"{server['protocol']}{server['hostname']}{server['location']}")

In [ ]:
print(f"The speed of light is {cc} {cc_units}.")

In [ ]:
print (obs_par["Fluxm0"])

---

In [ ]:
%%script bash
cat > config.yaml

What if we could put all those settings or constants in a separate file that is human readable and easy to maintain?

``` yaml
APOD Server:
    protocol :    https://
    hostname :    apod.nasa.gov
    location :    /apod/astropix.html
    search   :    /cgi-bin/apod/apod_search

use_anonymous: True

Constants:
    Speed of light:
        value: 299792458.
        unit : m s-1
        
    Speed of light in air:
        value: 299709348.         # cc/1.0002773, T=15C, standard atmospheric pressure
        unit : m s-1
        
    Solar Mass:                   # Harmanec & Prsa 2011
        value: 1.988547e30      
        unit : kg
    
    Solar Radius:                 # Harmanec & Prsa 2011
        value: 6.95508e8
        unit : m
    
    Solar Luminosity:             # Harmanec & Prsa 2011
        value: 3.846e26
        unit : kg m2 s-3
        
    Solar Effective Temperature:  # Harmanec & Prsa 2011
        value: 5779.5747
        unit : K

    Planck: [6.626070040E-34, "J s"]
    
ObservingParameters:

    MissionDuration:   6.5        # Total duration of the mission [yr], relevant for BOL->EOL degradation
    NumExposures   :   10         # Number of exposures
    BeginExposureNr:    0         # Sequential number of first exposure. useful for slurm parallellisation
    ExposureTime   :   21         # [s]
    RApointing     :   180        # Platform (not telescope) right ascension pointing coordinate      [deg] 
    DecPointing    :   -70        # Platform (not telescope) declination pointing coordinate          [deg]
    Fluxm0         :   1.00179e8  # Photon flux of a V=0 G2V-star                           [phot/s/m^2/nm]
    StarCatalogFile:   inputfiles/starcatalog.txt

```

In [ ]:
import yaml

In [ ]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [ ]:
cfg

In [ ]:
const = cfg["Constants"]
obs_par = cfg["ObservingParameters"]
server = cfg["APOD Server"]

In [ ]:
print (f"{server['protocol']}{server['hostname']}{server['location']}")

In [ ]:
cc = const["Speed of light"]["value"]
cc_units = const["Speed of light"]["unit"]

print (f"The speed of light is {cc} {cc_units}.")

In [ ]:
print (f"Planck's constant: {const['Planck'][0]} {const['Planck'][1]}")

In [ ]:
print (type(obs_par['Fluxm0']))
print (type(const['Planck'][0]))
print (type(const['Solar Mass']['value']))

There are two other implementations that solve some of the problems of the default package. Both implementations have their benefits and they work similar to the standard implementation. The ``ruamel.yaml`` has one additional trump. This latter has a round-trip that preserves comments in YAML files.

## ruamel.yaml

In [ ]:
from ruamel.yaml import YAML

In [ ]:
yml = YAML(typ='safe')
with open("config.yaml", 'r') as ymlfile:
    cfg = yml.load(ymlfile)

In [ ]:
const = cfg["Constants"]
obs_par = cfg["ObservingParameters"]
server = cfg["APOD Server"]

In [ ]:
cfg

In [ ]:
cfg["ObservingParameters"]

In [ ]:
print (type(obs_par['Fluxm0']))
print (type(const['Planck'][0]))
print (type(const['Solar Mass']['value']))

For using this in production code, you should of course always check and catch exceptions.

In [ ]:
with open("config.yaml", 'r') as ymlfile:
    try:
        cfg = yaml.load(ymlfile)
    except yaml.YAMLError as exc:
        logging.error(exc)

In [ ]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)


<div style="text-align: right"><button>[⇧ Go to top ⇧](#top_cell)</button></div>